# Cryptocurrency Data Pipeline with Snowflake & Snowpark

* Author: Sicheng Bao (@Jellysillyfish13), Yung Rou Ko (@KoYungRou), Anuj Rajendraprasad Nene (@Neneanuj)
* Last Updated: 2/28/2025

Welcome to the beginning of the Quickstart! Please refer to [the official Snowflake Notebook Data Engineering Quickstart](https://quickstarts.snowflake.com/guide/data_engineering_with_notebooks/index.html?index=..%2F..index#0) for all the details including set up steps.

## Step1. Setup Snowflake

During this step we will create our demo environment. Update the SQL variables below with your GitHub username and Personal Access Token (PAT) as well as with your forked GitHub repository information.
 

In [ ]:
USE ROLE ACCOUNTADMIN;

-- ----------------------------------------------------------------------------
-- Step #1: Create and assign role for Crypto Project
-- ----------------------------------------------------------------------------
SET MY_USER = CURRENT_USER();
CREATE OR REPLACE ROLE CRYPTO_ROLE;
GRANT ROLE CRYPTO_ROLE TO ROLE SYSADMIN;
GRANT ROLE CRYPTO_ROLE TO USER IDENTIFIER($MY_USER);

GRANT EXECUTE TASK ON ACCOUNT TO ROLE CRYPTO_ROLE;
GRANT MONITOR EXECUTION ON ACCOUNT TO ROLE CRYPTO_ROLE;
GRANT IMPORTED PRIVILEGES ON DATABASE SNOWFLAKE TO ROLE CRYPTO_ROLE;

-- ----------------------------------------------------------------------------
-- Step #2: Create Database and Warehouse for Crypto Project
-- ----------------------------------------------------------------------------
CREATE OR REPLACE DATABASE CRYPTO_DB;
GRANT OWNERSHIP ON DATABASE CRYPTO_DB TO ROLE CRYPTO_ROLE;

CREATE OR REPLACE WAREHOUSE CRYPTO_WH 
    WAREHOUSE_SIZE = XSMALL 
    AUTO_SUSPEND = 300 
    AUTO_RESUME = TRUE;
GRANT OWNERSHIP ON WAREHOUSE CRYPTO_WH TO ROLE CRYPTO_ROLE;

USE ROLE CRYPTO_ROLE;
USE WAREHOUSE CRYPTO_WH;
USE DATABASE CRYPTO_DB;
 
-- Schemas
CREATE OR REPLACE SCHEMA EXTERNAL;
CREATE OR REPLACE SCHEMA RAW;
CREATE OR REPLACE SCHEMA RAW_POS;
CREATE OR REPLACE SCHEMA HARMONIZED;
CREATE OR REPLACE SCHEMA ANALYTICS;

## Step2. raw_data_setup

1.  Create a CSV file format (skip the first two header lines)
2. Create an external stage to load cryptocurrency data (CSV file stored in S3)
3. Create the target table for storing raw-layer cryptocurrency data
Assuming the CSV file columns are ordered as follows:
Date, BTC_CLOSE, DOGE_CLOSE, ETH_CLOSE, BTC_HIGH, DOGE_HIGH, ETH_HIGH, BTC_LOW, DOGE_LOW, ETH_LOW, BTC_OPEN, DOGE_OPEN, ETH_OPEN, BTC_VOLUME, DOGE_VOLUME, ETH_VOLUME

In [ ]:
from snowflake.snowpark import Session
import pandas as pd
import yfinance as yf

# Snowflake connection
connection_parameters = {
    "account": "your_snowflake_account",
    "user": "your_username",
    "password": "your_password",
    "warehouse": "CRYPTO_WH",
    "database": "CRYPTO_DB",
    "schema": "RAW_CRYPTODB"
}

session = Session.builder.configs(connection_parameters).create()

# Fetch Crypto Prices
def fetch_crypto_prices():
    cryptos = ["BTC-USD", "ETH-USD", "DOGE-USD"]
    data = {crypto: yf.download(crypto, start="2020-01-01") for crypto in cryptos}
    
    df_list = []
    for symbol, df in data.items():
        df["symbol"] = symbol
        df = df.reset_index()[["Date", "symbol", "Close"]]
        df.columns = ["date", "symbol", "price"]
        df_list.append(df)
    
    return pd.concat(df_list)

# Load data into Snowflake
df = fetch_crypto_prices()
session.write_pandas(df, "CRYPTO_PRICES", auto_create_table=True)
